In [1]:
import torch
torch.cuda.is_available()

True

In [3]:
# yolo_opencv_face_detect.py
# Requires: pip install ultralytics opencv-python
# Replace 'yolov8n-face.pt' with the path to your face-specific YOLO weights.
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO model (use a face-detection weight; change path if needed)
model = YOLO('yolov11n-face.pt')

# Open default camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise SystemExit("Cannot open camera")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Run model on the BGR frame (Ultralytics accepts numpy arrays)
        results = model(frame)[0]  # results is a single Result object

        # Extract boxes, confidences, classes
        if results.boxes:
            xyxy = results.boxes.xyxy.cpu().numpy()    # shape (N,4)
            confs = results.boxes.conf.cpu().numpy()   # shape (N,)
            clss = results.boxes.cls.cpu().numpy()     # shape (N,)
            names = results.names                       # dict/list of class names

            for (x1, y1, x2, y2), conf, cls in zip(xyxy, confs, clss):
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                label = names[int(cls)] if int(cls) in names else str(int(cls))
                text = f"{label} {conf:.2f}"
                # Draw box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, text, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Show frame
        cv2.imshow("YOLO Face Detection", frame)

        # Quit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()


0: 480x640 2 faces, 61.0ms
Speed: 38.7ms preprocess, 61.0ms inference, 50.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 24.9ms
Speed: 4.0ms preprocess, 24.9ms inference, 14.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 faces, 22.5ms
Speed: 2.0ms preprocess, 22.5ms inference, 19.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 23.1ms
Speed: 2.8ms preprocess, 23.1ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 faces, 17.7ms
Speed: 3.4ms preprocess, 17.7ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 faces, 30.7ms
Speed: 2.5ms preprocess, 30.7ms inference, 13.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 faces, 16.1ms
Speed: 2.2ms preprocess, 16.1ms inference, 5.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.9ms
Speed: 1.7ms preprocess, 13.9ms inference, 14.2ms postprocess per image at shape (1, 3, 480, 64